In [16]:
# Import the required modules and libraries:

import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import numpy as np
import torchvision
from torchvision import datasets, models, transforms

In [17]:
#os.environ['CUDA_LAUNCH_BLOCKING']=1

In [18]:
class NeuralNetwork(nn.Module):
  def __init__(self):
    super(NeuralNetwork, self).__init__()
    self.conv1 = nn.Conv2d(3, 32, 3, padding = 1)
    self.conv2 = nn.Conv2d(32, 64, 3, padding = 1)
    self.conv3 = nn.Conv2d(64, 128, 3, padding = 1)
    self.pool = nn.MaxPool2d(2, 2)
    self.globalPool = nn.AdaptiveAvgPool2d((1, 1))
    self.fc1 = nn.Linear(128 * 4 * 4, 512)
    self.fc2 = nn.Linear(512, 100)
    self.relu = nn.ReLU()

  def forward(self, x):
    x = self.pool(self.relu(self.conv1(x)))
    x = self.pool(self.relu(self.conv2(x)))
    x = self.pool(self.relu(self.conv3(x)))
    x = x.view(-1, 128 * 4 * 4)
    x = self.relu(self.fc1(x))
    x = self.fc2(x)
    return x

model = NeuralNetwork()

In [19]:
# Resize the size of all images to a unanimous value (224, 224).
# Convert PIL image objects into Tensors.
# Normalize the tensor values based on the mean and standard deviation of the RGB values of all the images:

# transforms.Resize((128, 128)),

data_transforms = transforms.Compose([
transforms.RandomHorizontalFlip(),
transforms.ToTensor(),
transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))
])

In [20]:
# Create an object of torchvision.datasets.CIFAR100 to get the training and testing set:

trainset = datasets.CIFAR100(root = "/content/drive/MyDrive/5_Cisc442_CompVision/PR2", train = True, transform = data_transforms, download=True)
testset = datasets.CIFAR100(root = "/content/drive/MyDrive/5_Cisc442_CompVision/PR2", train=False, transform = data_transforms, download=True)

Files already downloaded and verified
Files already downloaded and verified


In [21]:
# Create a data loader.

data = torch.utils.data.DataLoader(trainset, batch_size=64, shuffle=True)
dataTestSet = torch.utils.data.DataLoader(testset, batch_size=64, shuffle=True)

In [22]:
# Set the number of epochs:
num_epochs = 30

In [23]:
# Move the model to GPU (if available):
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
model = model.to(device)

In [24]:
# Define a loss function for evaluating the trained model:
criterion = nn.CrossEntropyLoss()

In [25]:
# Create an optimizer with an initial learning rate and momentum:
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

In [26]:
# Create a scheduler to control the way that learning rate changes during the training process:
scheduler = lr_scheduler.StepLR(optimizer, step_size=7, gamma=0.1)

In [27]:
model.train()

# Iterate over the epochs and save the best model weights.
for i in range(num_epochs):
  for batch in data:

    # In every iteration, we get a mini-batch of images and their corresponding labels.
    # data is a mini-batch input
    images, labels = batch # data is a mini-batch input
    images = images.to(device)
    labels = labels.to(device)

    # Use zero_grad() to reset the calculated gradients.
    optimizer.zero_grad()

    # Use the current model weights for predication and backpropagate the prediction loss.
    outputs = model(images) # here the model is the pretrained VGG16
    loss = criterion(outputs, labels)
    loss.backward()
    optimizer.step()

  # After iterating over all batches and if we are in the training phase, we need to run scheduler.step() to update the scheduler status.
  scheduler.step()

In [28]:
# Save the model weights:
torch.save(model.state_dict(), 'best_model.pth')

In [29]:
# The testing process is very similar to the training process except that there is no need to backpropagate the loss.
# For testing the model, first, you need to prepare the model in the same way that we prepared it for the
# training process and load the best model that we saved in the training process.
model.load_state_dict(torch.load('best_model.pth'))

<All keys matched successfully>

In [30]:
total = 0
trues = 0

# After loading the model weights, set the model to evaluation mode.
model.eval()

# Then go through the test set
for batch in dataTestSet:
  images, labels = batch
  images = images.to(device)
  labels = labels.to(device)

  outputs = model(images)

  # predict the category of images
  _, preds = torch.max(outputs, 1)

  # compute the number of correctly classified images and the accuracy
  # correctClassImgs += (outputs == labels).sum().item()
  x = torch.eq(labels, preds)
  for a in range(len(x)):
    total += 1
    if x[a] == True:
      trues += 1


# accuracy = correct / total
accuracy = (trues / total)*100

# Finally, print the accuracy.
print(accuracy)

30.959999999999997
